In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import wandb
from tensorflow.keras import layers
from tensorflow.keras import backend

In [ ]:
CLASS_NAMES = np.array([])
for i in range(10):
    CLASS_NAMES =np.append(CLASS_NAMES,chr(ord("0")+i))
for i in range(26):
    CLASS_NAMES = np.append(CLASS_NAMES,chr(ord("A")+i))

In [ ]:
#functions
def parse_label(strIN):
    return (strIN==CLASS_NAMES).astype(float)
def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
    img = tf.io.decode_jpeg(a)
    img = tf.image.convert_image_dtype(img, dtype=tf.float16)/255.
    img = tf.image.per_image_standardization(img)
    return img

In [ ]:
#training data 01
def prepData01():  
  train_dir_data01 = os.path.abspath(os.getcwd())+"/train/data01_train/"
  data = pd.read_csv("./train/data01_train.csv")
  for i in range(len(data)):
      data.iloc[i,1] = list(data.iloc[i,1])
  arr = np.zeros([6,50000],str)
  for j in range(6):
      for i in range(len(data)):
          arr[j][i] = data.iloc[i,1][j]
  data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

  for i in range(6):
      data.iloc[:,i+1] = data.iloc[:,i+1].map(parse_label)

  TL = tf.convert_to_tensor(list(data.iloc[:,1].values))
  TL2 = tf.convert_to_tensor(list(data.iloc[:,2].values))
  TL3 = tf.convert_to_tensor(list(data.iloc[:,3].values))
  TL4 = tf.convert_to_tensor(list(data.iloc[:,4].values))
  TL5 = tf.convert_to_tensor(list(data.iloc[:,5].values))
  TL6 = tf.convert_to_tensor(list(data.iloc[:,6].values))

  train_dir = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False) 
  train_data = train_dir.map(lambda x: readimg_to_tensor(x))
  train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
  train_data01=tf.data.Dataset.zip((train_data,train_label))
  validation_split=0.1
  num_elements=50000
  split = int(num_elements*validation_split)
  train_data_gen=iter(train_data01.shuffle(50000,seed = 102).batch(50000))

  img , lb = next(train_data_gen)
  img_train = img[:-split]
  lb_train = (lb[0][:-split],lb[1][:-split],lb[2][:-split],lb[3][:-split],lb[4][:-split],lb[5][:-split])
  img_test = img[-split:]
  lb_test = (lb[0][-split:],lb[1][-split:],lb[2][-split:],lb[3][-split:],lb[4][-split:],lb[5][-split:])
  return img_train, lb_train, img_test, lb_test

In [ ]:
#choose data
img_train, lb_train, img_test, lb_test= prepData01()

In [ ]:
filepath ="h5s/" #"/content/drive/Shared drives/TingWeis_Drive/Colab Notebooks/Saved Model/"#

In [ ]:
config_default = {
    "weight_regu":1e-6,
    "LR":1e-3,
    "dropout":0.3,
    "name":"Mymodel",
    "batch":256
}

In [ ]:
#Slyne config
def Slyne_model(config=config_default):
    initializer = tf.keras.initializers.he_normal( seed = 3)
    alpha = config["weight_regu"]  # weight decay coefficient
    regularizer = tf.keras.regularizers.l2(alpha)
    dropout_rate =config["dropout"]
    inn = Input((60, 200, 3))
    out = inn
    out = Conv2D(filters=64, kernel_size=(3, 3), padding='valid', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = Conv2D(filters=64, kernel_size=(3, 3), padding='same', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(dropout_rate)(out)

    out = Conv2D(filters=128, kernel_size=(3, 3), padding='valid', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = Conv2D(filters=128, kernel_size=(3, 3), padding='same', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(dropout_rate)(out)

    out = Conv2D(filters=256, kernel_size=(3, 3), padding='valid', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = Conv2D(filters=256, kernel_size=(3, 3), padding='same', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(dropout_rate)(out)

    out = Conv2D(filters=512, kernel_size=(3, 3), padding='valid', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = Conv2D(filters=512, kernel_size=(3, 3), padding='same', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = layers.GlobalAveragePooling2D()(out)
    out = Dropout(dropout_rate)(out)
    
    out = layers.RepeatVector(6)(out)
    sep = layers.GRU(128,return_sequences=True)(out)
    sep0 = layers.Lambda(lambda x: x[:, 0, :])(sep)
    sep1 = layers.Lambda(lambda x: x[:, 1, :])(sep)
    sep2 = layers.Lambda(lambda x: x[:, 2, :])(sep)
    sep3 = layers.Lambda(lambda x: x[:, 3, :])(sep)
    sep4 = layers.Lambda(lambda x: x[:, 4, :])(sep)
    sep5 = layers.Lambda(lambda x: x[:, 5, :])(sep)
    dig1 = layers.Dense(36, name='digit1', activation='softmax')(sep0)
    dig2 = layers.Dense(36, name='digit2', activation='softmax')(sep1)
    dig3 = layers.Dense(36, name='digit3', activation='softmax')(sep2)
    dig4 = layers.Dense(36, name='digit4', activation='softmax')(sep3)
    dig5 = layers.Dense(36, name='digit5', activation='softmax')(sep4)
    dig6 = layers.Dense(36, name='digit6', activation='softmax')(sep5)
    model = tf.keras.models.Model(inputs=inn, outputs=[dig1, dig2,dig3,dig4,dig5,dig6],name="test")
    model.compile(
            loss=[
                tf.keras.losses.CategoricalCrossentropy(),
                tf.keras.losses.CategoricalCrossentropy(),
                tf.keras.losses.CategoricalCrossentropy(),
                tf.keras.losses.CategoricalCrossentropy(),
                tf.keras.losses.CategoricalCrossentropy(),
                tf.keras.losses.CategoricalCrossentropy(),
            ],
            optimizer=tf.keras.optimizers.Adam(learning_rate=myconfig["LR"], beta_1=0.9),
            metrics=['accuracy'])
    return model

In [ ]:
def trainModel(img_train, lb_train,callB,epochs=100,config = config_default):
  history = model.fit(
        img_train,
        lb_train,
        batch_size = config["batch"],
        shuffle = True,
        epochs=epochs,
        verbose=1,
        callbacks = callB
        )
def switch_to_SGD():
  model.compile(
    loss=[
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy()
    ],
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9), 
    metrics=['accuracy'])
class myCallback(tf.keras.callbacks.Callback):
  def __init__(self, config=config_default,**kwargs):
    self.config = config
    self.best = 0
    self.epochs = 0
    self.wait = 0
    self.reduce_once = False
    self.patient = 8  #wait 8 epochs for early stopping
  def on_epoch_end(self, epoch, logs=None):
    self.epochs+=1
    b= np.prod([logs['digit1_accuracy'],
              logs['digit2_accuracy'],
              logs['digit3_accuracy'],
              logs['digit4_accuracy'],
              logs['digit5_accuracy'],
              logs['digit6_accuracy'],
              ])
    result = self.model.predict(img_test)
    per_digit_acc,val_acc_all = allright(result,lb_test)
    digit_acc = np.prod(per_digit_acc)
    wandb.log({"loss":logs["loss"],"accuracy": b, "epoch": self.epochs, "val_accuracy_all":val_acc_all, "val_accuracy":digit_acc})
    print("\nepoch: {}, loss: {:.4f}, accuracy: {:.5f}, val_accuracy_all: {:.5f}, val_accuracy: {:.5f}, val_per_digit_accuracy: {}".format(
        self.epochs,  logs["loss"],   b,val_acc_all,    digit_acc,    per_digit_acc))
    if val_acc_all> self.best:
      print("                            ***** accuracy improved from {:.5f} to {:.5f}!! *****".format(self.best,val_acc_all))
      print("Model saved to "+filepath+self.config["name"]+".hdf5\n" )
      self.best = val_acc_all
      self.model.save(filepath+self.config["name"]+".hdf5")
      self.wait = 0
      wandb.run.summary["best_val_accuracy_all"] = val_acc_all
      wandb.run.summary["best_epoch"] = self.epochs
    elif self.epochs>30:
      if self.wait>=self.patient:   #switch to SGD or reduce lr if already on SGD 
        if hasattr(self.model.optimizer,'momentum') and not self.reduce_once:     #reduce lr on SGD if not done it once yet
          self.model.optimizer.lr=self.model.optimizer.lr/2
          print("\n\n Change leraning rate to {}, continued ... \n".format(self.model.optimizer.lr))
          self.wait=0
          self.reduce_once=True
        else:
          self.model.stop_training = True
          self.wait=0
        print("Model is not learning, training stop at {}".format(epoch))
      print("Model accuracy did not improve from {:.4f}\n".format(self.best))
      self.wait+=1
      
class Switch_SGD_callback(tf.keras.callbacks.Callback):
  def __init__(self,on_train_end,config=config_default, **kwargs):
    self.do_on_train_end=on_train_end
    self.config = config
  def on_train_end(self, logs=None):
      self.do_on_train_end(self.config)

def do_after_train(config=config_default):
    '''do this after a seesion is over'''
    print("\n ******** Switching to SGD!! **********")
    switch_to_SGD()
    trainModel(img_train, lb_train,[myCB],config=myconfig)

def allright(inputdata,label):
    '''calculate the accuracy of each digit and of all right'''
    a = np.argmax(inputdata,axis = 2)
    b = np.argmax(lb_test[0],axis=1)
    b = np.expand_dims(b,0)
    for i in range(5):
        b = np.concatenate((b,np.expand_dims(np.argmax(lb_test[i+1],axis=1),0)),axis=0)
    acc_digit = np.mean(a==b,axis=1)
    acc_all = np.mean(np.all(a.transpose()==b.transpose(),axis=1))
    return acc_digit,acc_all 

In [ ]:
myconfig = {
    "stack" : "conv 3x3 64-512 GRU 128 ",
    "weight_regu" : 1e-5,
    "dropout" : 0.2,
    "name" : "slyneGA_GRU2",
    "output_WR" : False,
    "total_params" : 5066520,
    "opt_schedule" : "1e-4 adam + 1e-4 SGD",
    "batch" : 128,
    "GlobalAvg":True,
    "Augmentation":False,
    "LR":1e-4
}

In [ ]:
wandb.init(project="my-project-railway",reinit=True,name=myconfig["name"],config=myconfig) 
#ini call back
myCB = myCallback(config = myconfig)
model=Slyne_model(config=myconfig)
trainModel(img_train, lb_train,[myCB,Switch_SGD_callback(do_after_train,config = myconfig)],config = myconfig)